In [1]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Device name:", torch.cuda.get_device_name(0))
    print("Device count:", torch.cuda.device_count())


PyTorch version: 2.10.0.dev20250928+cu130
CUDA available: True
Device name: NVIDIA GeForce RTX 5060 Ti
Device count: 1


In [1]:
# Creating a reusable Python function to draw a colorful, publication-ready flowchart.
# The function will draw rounded boxes, arrows, and support multiline text. It saves PNG and SVG outputs
# suitable for embedding in a research paper (high DPI PNG and vector SVG).
# It will also draw the specific flowchart from the user's image as an example.

import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch
import matplotlib as mpl
import textwrap
import os

mpl.rcParams['font.family'] = 'DejaVu Sans'  # widely available font that's readable in figures

def draw_box(ax, center, width, height, text, bgcolor='#a6cee3', edgecolor='#1f78b4', fontsize=10, pad=6):
    """Draw a rounded rectangle with centered multiline text."""
    x, y = center
    box = FancyBboxPatch((x - width/2, y - height/2),
                         width, height,
                         boxstyle="round,pad=0.3,rounding_size=6",
                         linewidth=1.6,
                         facecolor=bgcolor,
                         edgecolor=edgecolor)
    ax.add_patch(box)
    # wrap text
    wrapped = "\n".join(textwrap.wrap(text, width=40))
    ax.text(x, y, wrapped, ha='center', va='center', fontsize=fontsize, wrap=True)
    return box

def draw_arrow(ax, p1, p2, rad=0.0, arrowstyle='-|>', linewidth=1.6, color='#333333'):
    """Draw an arrow from p1 to p2. p1 and p2 are (x,y) coords."""
    arrow = FancyArrowPatch(p1, p2, connectionstyle=f"arc3,rad={rad}",
                            arrowstyle=arrowstyle, mutation_scale=18,
                            linewidth=linewidth, color=color)
    ax.add_patch(arrow)
    return arrow

def create_flowchart(filename_prefix='/mnt/data/flowchart_research', dpi=300, figsize=(8,12)):
    """
    Create and save a colorful flowchart representing the provided methodology.
    Returns file paths for PNG and SVG.
    """
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 14)
    ax.axis('off')

    # Colors palette (soft, publication-friendly)
    colors = {
        'ingest': '#fed9a6',         # warm
        'preproc': '#b2df8a',        # green
        'corr': '#a6cee3',           # blue
        'feat': '#fdbf6f',           # orange
        'split': '#c7c7c7',          # gray
        'model': '#b3b3ff',          # purple
        'train': '#ffffb3',          # yellow
        'eval': '#ffb3b3',           # pink
        'results': '#c2f0c2',        # mint
        'deploy': '#d9d9d9',         # light gray
    }

    # Define box positions and sizes (center coordinates)
    boxes = {}
    h = 0.9  # height for most boxes
    w = 7.0  # width for main wide boxes

    boxes['Data Ingestion'] = {'center': (5, 13), 'w': w, 'h': h, 'color': colors['ingest'],
                               'text': "Data Ingestion\n(Kaggle dataset: 20,745 × 44)"}

    boxes['Preprocessing'] = {'center': (5, 11.5), 'w': w, 'h': 1.5, 'color': colors['preproc'],
                              'text': "Preprocessing\n• Missing values: median/mode\n• Encoding: label / one-hot\n• Scaling: Min–Max & Robust Standardization"}

    # Two side-by-side boxes after preprocessing
    boxes['Correlation & Redundancy'] = {'center': (2.5, 9.5), 'w': 4.0, 'h': 1.1, 'color': colors['corr'],
                                         'text': "Correlation & Redundancy\n• Pearson / Cramér's V\n• VIF filtering\n• Drop collinear features"}

    boxes['Feature Engineering'] = {'center': (7.5, 9.5), 'w': 4.0, 'h': 1.1, 'color': colors['feat'],
                                    'text': "Feature Engineering\n• Symptom Count\n• Test interaction scores\n• Risk Index"}

    boxes['Stratified Split'] = {'center': (5, 7.8), 'w': w, 'h': h, 'color': colors['split'],
                                 'text': "Stratified Split\n80% train / 10% val / 10% test"}

    boxes['Custom Dual-Channel CNN'] = {'center': (5, 6.0), 'w': w, 'h': 1.4, 'color': colors['model'],
                                        'text': "Custom Dual-Channel CNN\n• Input: (44, 1, 1)\n• Conv channels + Pool + BN\n• Fusion: Average\n• Dense layers + Dropout\n• Softmax output"}

    boxes['Training'] = {'center': (5, 4.8), 'w': w, 'h': 1.0, 'color': colors['train'],
                         'text': "Training\n• Adam (1e-3), batch 16\n• Epochs: 25 (~45 min)\n• EarlyStopping on val_loss"}

    boxes['Evaluation'] = {'center': (2.8, 3.2), 'w': 4.2, 'h': 1.0, 'color': colors['eval'],
                           'text': "Evaluation\n• Accuracy, Precision, Recall, F1\n• ROC–AUC\n• Confusion Matrices"}

    boxes['Results'] = {'center': (7.2, 3.2), 'w': 4.2, 'h': 1.0, 'color': colors['results'],
                        'text': "Results\n• Genetic Disorder: 99.95%\n• Subclass: 99.91%\n• Stable curves"}

    boxes['Deployment & Reporting'] = {'center': (5, 1.4), 'w': w, 'h': 1.0, 'color': colors['deploy'],
                                       'text': "Deployment & Reporting\n• Model export\n• Figures\n• Interpretability (future: SHAP / LRP)"}

    # Draw boxes
    for name, props in boxes.items():
        draw_box(ax, props['center'], props['w'], props['h'], props['text'],
                 bgcolor=props['color'], edgecolor='#333333', fontsize=9)

    # Draw arrows to connect boxes logically
    draw_arrow(ax, (5, 12.2), (5, 12.0))  # small connector from ingestion to preprocessing
    draw_arrow(ax, (5, 10.7), (5, 10.0))  # preproc to split area (but visually we'll go to side boxes first)
    # Preproc to Correlation & Feature boxes
    draw_arrow(ax, (4.1, 10.0), (2.5, 10.6), rad=0.15)
    draw_arrow(ax, (5.9, 10.0), (7.5, 10.6), rad=-0.15)
    # Correlation & Feature to Stratified Split
    draw_arrow(ax, (2.5, 8.95), (2.8, 8.0), rad=0.0)
    draw_arrow(ax, (7.5, 8.95), (7.2, 8.0), rad=0.0)
    # Arrows from those points into central split box
    draw_arrow(ax, (2.8, 8.0), (4.6, 7.9), rad=0.0)
    draw_arrow(ax, (7.2, 8.0), (5.4, 7.9), rad=0.0)

    # Stratified split to model
    draw_arrow(ax, (5, 6.95), (5, 6.4))

    # Model to training
    draw_arrow(ax, (5, 5.35), (5, 5.05))

    # Training to Evaluation and Results (two branches)
    draw_arrow(ax, (4.0, 4.55), (2.8, 3.9), rad=0.18)
    draw_arrow(ax, (6.0, 4.55), (7.2, 3.9), rad=-0.18)

    # Evaluation & Results to Deployment
    draw_arrow(ax, (2.8, 2.2), (4.6, 1.7))
    draw_arrow(ax, (7.2, 2.2), (5.4, 1.7))

    # Title
    ax.text(5, 13.6, "Methodology Flowchart", ha='center', va='center', fontsize=14, fontweight='bold')

    # Save outputs (high DPI PNG & vector SVG)
    png_path = filename_prefix + '.png'
    svg_path = filename_prefix + '.svg'
    plt.savefig(png_path, dpi=dpi, bbox_inches='tight', facecolor='white')
    plt.savefig(svg_path, bbox_inches='tight', facecolor='white')
    plt.close(fig)

    return png_path, svg_path

# Create the flowchart and save files
png_file, svg_file = create_flowchart()

# Show the PNG inline as a confirmation (Jupyter will render this)
from IPython.display import Image, display
display(Image(filename=png_file))

# Print file locations so the outer chat can reference them
png_file, svg_file, os.path.exists(png_file), os.path.exists(svg_file)


ModuleNotFoundError: No module named 'matplotlib'